== This week's links ==

* [https://youtu.be/V2h3IOBDvrA The lesson video]
* [http://wiki.fast.ai/index.php/Lesson_4_Notes Lesson 4 notes]
* [[Lesson 4 Timeline]]


In [1]:
%matplotlib inline
import utils
import imp
imp.reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [30]:
path = 'data/ml-latest-small/'
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size = 64

## Setup data

We're working with the movielens data, which contains one rating per row, like this:

In [6]:
ratings = pd.read_csv(path + 'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
len(ratings)

100004

Just for displat purposes, let's read in the movie names too.

In [8]:
movie_names = pd.read_csv(path + 'movies.csv').set_index('movieId')['title'].to_dict()

In [10]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [11]:
userid2idx = {o:i for i, o in enumerate(users)}
movieid2idx = {o:i for i, o in enumerate(movies)}

We update the move and use ids so that they are contiguous integers, which we want when using embeddings.

In [12]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [13]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [14]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

This is the number of latent facors in each embedding.

In [15]:
n_factors = 50
np.random.seed = 42

Randomly split into training and validation.

In [16]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

## Create subset for Excel

We create a crostab of the most popular movies and most movie-addicted users which we'll copy into excel for creating a simple example. This is not necessary for any of the modeling below however.

In [17]:
g = ratings.groupby('userId')['rating'].count()
topUsers = g.sort_values(ascending = False)[:15]

In [18]:
g = ratings.groupby('movieId')['rating'].count()
topMovies = g.sort_values(ascending = False)[:15]

In [19]:
top_r = ratings.join(topUsers, rsuffix = '_r', how = 'inner', on = 'userId')

In [21]:
top_r = top_r.join(topMovies, rsuffix = '_r', how = 'inner', on = 'movieId')

In [22]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc = np.sum)

movieId,27,49,57,72,79,89,92,99,143,179,180,197,402,417,505
userId,,,,,,,,,,,,,,,
14,3.0,5.0,1.0,3.0,4.0,4.0,5.0,2.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0
29,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0
72,4.0,5.0,5.0,4.0,5.0,3.0,4.5,5.0,4.5,5.0,5.0,5.0,4.5,5.0,4.0
211,5.0,4.0,4.0,3.0,5.0,3.0,4.0,4.5,4.0,NaN,3.0,3.0,5.0,3.0,NaN
212,2.5,NaN,2.0,5.0,NaN,4.0,2.5,NaN,5.0,5.0,3.0,3.0,4.0,3.0,2.0
293,3.0,NaN,4.0,4.0,4.0,3.0,NaN,3.0,4.0,4.0,4.5,4.0,4.5,4.0,NaN
310,3.0,3.0,5.0,4.5,5.0,4.5,2.0,4.5,4.0,3.0,4.5,4.5,4.0,3.0,4.0
379,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,4.0,NaN,3.0,5.0,4.0,4.0
451,4.0,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,2.0,3.5,5.0


## Dot product

The most basic model is a dot product of a movie embedding and a user embedding. Let's see how well that works:

In [24]:
user_in = Input(shape = (1,), dtype = 'int64', name = 'user_in')
u = Embedding(n_users, n_factors, input_length = 1, embeddings_regularizer = l2(1e-4))(user_in)
movie_in = Input(shape = (1,), dtype = 'int64', name = 'movie_in')
m = Embedding(n_movies, n_factors, input_length = 1, embeddings_regularizer = l2(1e-4))(movie_in)

In [28]:
x = merge([u, m], mode = 'dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss = 'mse')

C:\Users\hjkim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
C:\Users\hjkim\Anaconda3\lib\site-packages\keras-2.0.8-py3.6.egg\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [31]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 1, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79626 samples, validate on 20378 samples
Epoch 1/1
79626/79626 [==============================] - 34s - loss: 4.7103 - val_loss: 3.8149


In [ ]:
model.optimizer.lr = 0.01

In [32]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 3, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79626 samples, validate on 20378 samples
Epoch 1/3
79626/79626 [==============================] - 34s - loss: 3.1011 - val_loss: 2.9437
Epoch 2/3
79626/79626 [==============================] - 35s - loss: 2.4783 - val_loss: 2.6467
Epoch 3/3
79626/79626 [==============================] - 35s - loss: 2.2552 - val_loss: 2.5680


In [33]:
model.optimizer.lr = 0.001

In [34]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 6, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79626 samples, validate on 20378 samples
Epoch 1/6
79626/79626 [==============================] - 35s - loss: 2.1775 - val_loss: 2.5417
Epoch 2/6
79626/79626 [==============================] - 34s - loss: 2.1443 - val_loss: 2.5386
Epoch 3/6
79626/79626 [==============================] - 34s - loss: 2.1218 - val_loss: 2.5427
Epoch 4/6
79626/79626 [==============================] - 35s - loss: 2.1018 - val_loss: 2.5466
Epoch 5/6
79626/79626 [==============================] - 35s - loss: 2.0795 - val_loss: 2.5511
Epoch 6/6
79626/79626 [==============================] - 34s - loss: 2.0566 - val_loss: 2.5623


The [best_benchmarks](http://www.librec.net/example.html) are a bit ober 0.9, so this model doesn't seem to be working that well...

## Bias

The problem is likely yo be taht we don't have bias terms - that is, a single bias for each user and each movie representing how positive or negatibe each user is, and how good each movie is. We can add that easily by simply creating an embedding with one output for each movie and each user, and addin git to our output.

In [37]:
def embedding_input(name, n_in, n_out, reg) :
    inp = Input(shape = (1,), dtype = 'int64', name = name)
    return inp, Embedding(n_in, n_out, input_length = 1, embeddings_regularizer = l2(reg))(inp)

In [38]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [41]:
def create_bias(inp, n_in) :
    x = Embedding(n_in, 1, input_length = 1)(inp)
    return Flatten()(x)

In [42]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [43]:
x = merge([u, m], mode = 'dot')
x = Flatten()(x)
x = merge([x, ub], mode = 'sum')
x = merge([x, mb], mode = 'sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss = 'mse')

C:\Users\hjkim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
C:\Users\hjkim\Anaconda3\lib\site-packages\keras-2.0.8-py3.6.egg\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\hjkim\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\hjkim\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use inst

In [44]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 1, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79626 samples, validate on 20378 samples
Epoch 1/1
79626/79626 [==============================] - 36s - loss: 8.8205 - val_loss: 3.5296


In [45]:
model.optimizer.lr = 0.01

In [46]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 6, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79626 samples, validate on 20378 samples
Epoch 1/6
79626/79626 [==============================] - 33s - loss: 2.5985 - val_loss: 2.3011
Epoch 2/6
79626/79626 [==============================] - 34s - loss: 2.0001 - val_loss: 2.0936
Epoch 3/6
79626/79626 [==============================] - 33s - loss: 1.8347 - val_loss: 2.0049
Epoch 4/6
79626/79626 [==============================] - 33s - loss: 1.7397 - val_loss: 1.9325
Epoch 5/6
79626/79626 [==============================] - 33s - loss: 1.6600 - val_loss: 1.8591
Epoch 6/6
79626/79626 [==============================] - 33s - loss: 1.5862 - val_loss: 1.7910


In [47]:
model.optimizer.lr = 0.001

In [48]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 10, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79626 samples, validate on 20378 samples
Epoch 1/10
79626/79626 [==============================] - 33s - loss: 1.5135 - val_loss: 1.7283
Epoch 2/10
79626/79626 [==============================] - 33s - loss: 1.4444 - val_loss: 1.6675
Epoch 3/10
79626/79626 [==============================] - 33s - loss: 1.3765 - val_loss: 1.6112
Epoch 4/10
79626/79626 [==============================] - 33s - loss: 1.3106 - val_loss: 1.5587
Epoch 5/10
79626/79626 [==============================] - 33s - loss: 1.2491 - val_loss: 1.5088
Epoch 6/10
79626/79626 [==============================] - 36s - loss: 1.1883 - val_loss: 1.4595
Epoch 7/10
79626/79626 [==============================] - 40s - loss: 1.1302 - val_loss: 1.4135s: 1
Epoch 8/10
79626/79626 [==============================] - 41s - loss: 1.0757 - val_loss: 1.3701
Epoch 9/10
79626/79626 [==============================] - 42s - loss: 1.0225 - val_loss: 1.3292
Epoch 10/10
79626/79626 [==============================] - 37s - loss: 0.9736 - va

In [49]:
model.optimizer.lr = 0.001

In [50]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 5, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79626 samples, validate on 20378 samples
Epoch 1/5
79626/79626 [==============================] - 35s - loss: 0.9263 - val_loss: 1.2561
Epoch 2/5
79626/79626 [==============================] - 36s - loss: 0.8828 - val_loss: 1.2249
Epoch 3/5
79626/79626 [==============================] - 37s - loss: 0.8416 - val_loss: 1.1980
Epoch 4/5
79626/79626 [==============================] - 36s - loss: 0.8040 - val_loss: 1.1721
Epoch 5/5
79626/79626 [==============================] - 36s - loss: 0.7688 - val_loss: 1.1461


This result is quite a bit better than the best benchmarks that we could find with a quick google search - so klooks like a greate approach!

In [51]:
model.save_weights(model_path + 'bias.h5')

In [52]:
model.load_weights(model_path + 'bias.h5')

We can use the model to generate predictions by passing a apir of ints - a user id and a movie id. For instance, this predicts that user #3 would really enjoy movie #6.

In [53]:
model.predict([np.array([3]), np.array([6])])

array([[ 5.0111]], dtype=float32)